In [5]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.4.0/spark-3.4.0-bin-hadoop3.tgz
!tar xf spark-3.4.0-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done

gzip: stdin: unexpected end of file
tar: Unexpected EOF in archive
tar: Unexpected EOF in archive
tar: Error is not recoverable: exiting now


Exception: ignored

In [ ]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [ ]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"



In [ ]:
# 2. Create a temporary view of the DataFrame.
df = spark.read.csv("./home_sales_revised.csv", sep=",", header=True, ignoreLeadingWhiteSpace=True, ignoreTrailingWhiteSpace=True)
df.createOrReplaceTempView('home_sales')
df.show(10)


In [ ]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
from pyspark.sql.functions import year, avg, format_number

# Filter by four-bedroom houses
four_bedroom_houses = df.filter(df['bedrooms'] == 4)

# Extract the year of the sale date and calculate the average price per year
average_price_per_year = four_bedroom_houses.groupBy(year('date').alias('year')) \
                            .agg(format_number(avg('price'), 2).alias('average_price')) \
                            .orderBy('year')
# Show results
average_price_per_year.show(15)


In [ ]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
# Filter by three-bedroom and three-bathroom houses
three_bedroom_and_bathroom_houses = df.filter((df['bedrooms'] == 3) & (df['bathrooms'] == 3))

# Group by the year the home was built and calculate the average price
average_price_per_build_year = three_bedroom_and_bathroom_houses.groupBy('date_built') \
                            .agg(format_number(avg('price'), 2).alias('average_price')) \
                            .orderBy('date_built')
# Show results
average_price_per_build_year.show()

In [ ]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
# Filtrar por casas de tres dormitorios, tres baños, dos pisos y mayor o igual a 2000 pies cuadrados
filtered_houses = df.filter(
    (df['bedrooms'] == 3) &
    (df['bathrooms'] == 3) &
    (df['floors'] == 2) &
    (df['sqft_living'] >= 2000)
)

average_price_per_year_construction = filtered_houses.groupBy('date_built') \
                            .agg(format_number(avg('price'), 2).alias('average_price')) \
                            .orderBy('date_built')

average_price_per_year_construction.show()

In [ ]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.
start_time = time.time()

home_filter = df.filter(df['price'] >= 350000)

# Agrupar por la calificación de "view" y calcular el precio promedio
average_price_per_view = home_filter.groupBy('view') \
                            .agg(format_number(avg('price'), 2).alias('average_price')) \
                            .orderBy('view')

average_price_per_view.show()

print("--- %s seconds ---" % (time.time() - start_time))

--- 7.200241088867188e-05 seconds ---


In [ ]:
# 7. Cache the the temporary table home_sales.
spark.catalog.cacheTable('home_sales')

In [ ]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

In [ ]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.
start_time = time.time()

filter_house_using_cache = spark.sql("""
    SELECT view, FORMAT_NUMBER(AVG(price), 2) as average_price
    FROM home_sales
    WHERE price >= 350000
    GROUP BY view
    ORDER BY view
""")
filter_house_using_cache.show()

print("--- %s seconds ---" % (time.time() - start_time))

# Delete table from cache
# spark.catalog.uncacheTable('home_sales')


--- 5.459785461425781e-05 seconds ---


In [ ]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
import os
output_path = "./save_data"
df.write.partitionBy("date_built").parquet(output_path, mode='overwrite')

In [ ]:
# 11. Read the parquet formatted data.
p_df = spark.read.parquet('p_sales')

In [ ]:
# 12. Create a temporary table for the parquet data.
p_df.createOrReplaceTempView('p_sales')

In [ ]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.
start_time = time.time()
start_time = time.time()

filter_house_using_parquet = spark.sql("""
    SELECT view, FORMAT_NUMBER(AVG(price), 2) as average_price
    FROM p_sales
    WHERE price >= 350000
    GROUP BY view
    ORDER BY view
""")

print("--- %s seconds ---" % (time.time() - start_time))

--- 7.104873657226562e-05 seconds ---


In [ ]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable('home_sales')

In [ ]:
# 15. Check if the home_sales is no longer cached
is_cached = spark.catalog.isCached('home_sales')